In [1]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

print("Tensorflow v" + tf.__version__)
print("Tensorflow Decision Forests V" + tfdf.__version__)
dataset_df = pd.read_csv('../data/train.csv')
print(dataset_df.head(5))

print("0" * 100)
print(dataset_df.columns)
print("2" * 100)
# There are 12 feature columns. Using these features your model has to predict whether the passenger is rescued or not indicated by the column Transported.
#
# add Codeadd Markdown
# Let us quickly do a basic exploration of the dataset
print(dataset_df.describe())
print(dataset_df.info())
# Bar chart for label column: Transported¶
plot_df = dataset_df.Transported.value_counts()
plot_df.plot(kind='bar')
# Numerical data distribution¶
# Let us plot all the numerical columns and their value counts:


# Prepare the dataset¶
# We will drop both PassengerId and Name columns as they are not necessary for model training.


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(10, 10))
plt.subplots_adjust(top=1)
sns.histplot(dataset_df['Age'], bins=50, color='b', ax=ax[0])
sns.histplot(dataset_df['FoodCourt'], bins=50, color='b', ax=ax[1])
sns.histplot(dataset_df['ShoppingMall'], bins=50, color='b', ax=ax[2])
sns.histplot(dataset_df['Spa'], bins=50, color='b', ax=ax[3])
sns.histplot(dataset_df['VRDeck'], bins=50, color='b', ax=ax[4])

In [ ]:
dataset_df.isnull().sum().sort_values(ascending=False)

In [ ]:
dataset_df = dataset_df.drop(['PassengerId', 'Name'], axis=1)
print("4" * 100)
print(dataset_df.head(5))

# plt.show()
# We will check for the missing values using the following code:
dataset_df.isnull().sum().sort_values(ascending=False)
# This dataset contains a mix of numeric, categorical and missing features. TF-DF supports all these feature types natively, and no preprocessing is required.
#
# But this datatset also has boolean fields with missing values. TF-DF doesn't support boolean fields yet. So we need to convert those fields into int. To account for the missing values in the boolean fields, we will replace them with zero.
#
# In this notebook, we will replace null value entries with zero for numerical columns as well and only let TF-DF handle the missing values in categorical columns.
#
# Note: You can choose to let TF-DF handle missing values in numerical columns if need be.
dataset_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = dataset_df[[
    'VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
]].fillna(value=0)

In [ ]:
dataset_df.Cabin.head()

In [ ]:

label = 'Transported'
dataset_df[label] = dataset_df[label].astype(int)
# We will also convert the boolean fields CryoSleep and VIP to int.
print("5" * 100)
dataset_df['VIP'] = dataset_df['VIP'].astype(int)
dataset_df['CryoSleep'] = dataset_df['CryoSleep'].astype(int)
# The value of column Cabin is a string with the format Deck/Cabin_num/Side. Here we will split the Cabin column and create 3 new columns Deck, Cabin_num and Side, since it will be easier to train the model on those individual data.
# Run the following command to split the column Cabin into columns Deck, Cabin_num and Side
dataset_df[[ "Deck", "Cabin_num", "Side"]]=dataset_df['Cabin'].str.split('/',expand=True)
# Remove original Cabin column from the dataset since it's not needed anymore.
print("6" * 100)
print(dataset_df.columns)
try:
    dataset_df.drop('Cabin',inplace=True,axis=1)
except KeyError:
    print('Field does not exists!')



In [ ]:
dataset_df.head(5)

In [ ]:
def split_dataset(dataset, test_ratio=0.2):
    test_indice = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indice], dataset[test_indice]


In [ ]:
label = 'Transported'
train_ds_pd, valid_ds_pd = split_dataset(dataset_df)
print("{} examples in training ,{} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label)

In [2]:
tfdf.keras.get_all_models()


NameError: name 'tfdf' is not defined

In [ ]:
rf = tfdf.keras.RandomForestModel(hyperparameter_template='benchmark_rank1')
rf = tfdf.keras.RandomForestModel()
rf.compile(metrics=['accuracy'])
rf.fit(x=train_ds)
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx=0, max_depth=3)
